# 필요한 패키지 다운받기

In [425]:
#내 환경에 한글폰트 설치(일단 나눔이라는 폰트 1개만)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent

In [426]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
#matplotlib의 폰트를 Nanum 폰트로 지정합니다

#하고나서 꼭 Ctrl + M을 눌러야함

In [427]:
# 데이터 불러오기(코랩 환경이라 google 마운트를 씁니다)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [428]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/dataton/data/')

In [429]:
!ls 

 진짜최종
'소상공인시장진흥공단_상가(상권)정보_서울_202203.csv'
'서울시 우리마을가게 상권분석서비스(상권영역).csv'
'서울시_우리마을가게_상권분석서비스(구_상권배후지_집객시설).csv'
 estimation_sales_data
 income_consumption_data
 Main컬럼
 map.html
 업종별매출_상권업종대분류.xlsx


# 필요한 라이브러리 import

In [430]:
import warnings
warnings.filterwarnings('ignore')

import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import time
from functools import reduce

!pip install pyproj
import pyproj

import json
import requests
%matplotlib inline 
import seaborn as sns 
sns.set()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [431]:
pwd

'/content/drive/MyDrive/Colab Notebooks/dataton/data'

In [432]:
!ls

 진짜최종
'소상공인시장진흥공단_상가(상권)정보_서울_202203.csv'
'서울시 우리마을가게 상권분석서비스(상권영역).csv'
'서울시_우리마을가게_상권분석서비스(구_상권배후지_집객시설).csv'
 estimation_sales_data
 income_consumption_data
 Main컬럼
 map.html
 업종별매출_상권업종대분류.xlsx


# 집객시설(지하철역, 버스정거장) 데이터 전처리

In [433]:
info = pd.read_csv("./Main컬럼/commercial_info_2021.csv", sep = ',', index_col = 0)
area = pd.read_csv("./Main컬럼/commercial_area2021.csv", sep = ',')
facility = pd.read_csv('./서울시_우리마을가게_상권분석서비스(구_상권배후지_집객시설).csv', sep=',', encoding='euc-kr')

In [434]:
com_list = [info, area, facility]

In [435]:
for i in com_list:
    print(i.columns)

Index(['상호명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드', '상권업종중분류명', '상권업종소분류코드',
       '상권업종소분류명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '경도',
       '위도'],
      dtype='object')
Index(['기준_년월_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '엑스좌표_값', '와이좌표_값',
       '시군구_코드'],
      dtype='object')
Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '집객시설_수', '관공서_수', '은행_수', '종합병원_수', '일반_병원_수', '약국_수', '유치원_수',
       '초등학교_수', '중학교_수', '고등학교_수', '대학교_수', '백화점_수', '슈퍼마켓_수', '극장_수',
       '숙박_시설_수', '공항_수', '철도_역_수', '버스_터미널_수', '지하철_역_수', '버스_정거장_수'],
      dtype='object')


In [436]:
print(facility.shape)
facility.head()

(31722, 26)


,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,집객시설_수,관공서_수,은행_수,종합병원_수,...,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수
0,2021,4,A,골목상권,2110939,양재역 8번(양재동우체국),23,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1,2021,4,A,골목상권,2110148,어린이대공원역 3번,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,4,R,전통시장,2130039,청계6가 지하도상가,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,4,R,전통시장,2130028,을지입구지하상가(을지로지하보도쇼핑센터),1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,2021,4,A,골목상권,2110231,장안교,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [437]:
# 2021년도 데이터만 추출

facility = facility[facility['기준_년_코드'] == 2021]
facility = facility[facility['기준_분기_코드'] == 4]
facility.shape

(1575, 26)

In [438]:
bus_row = ['상권_코드', '버스_정거장_수']
subway_row = ['상권_코드', '지하철_역_수']

In [439]:
facility.isnull().sum()

기준_년_코드          0
기준_분기_코드         0
상권_구분_코드         0
상권_구분_코드_명       0
상권_코드            0
상권_코드_명          0
집객시설_수           0
관공서_수          938
은행_수          1082
종합병원_수        1550
일반_병원_수       1441
약국_수           557
유치원_수         1416
초등학교_수        1562
중학교_수         1571
고등학교_수        1564
대학교_수         1486
백화점_수         1564
슈퍼마켓_수        1468
극장_수          1574
숙박_시설_수       1429
공항_수          1575
철도_역_수        1575
버스_터미널_수      1572
지하철_역_수       1379
버스_정거장_수       294
dtype: int64

In [440]:
bus = facility[bus_row]
subway = facility[subway_row]

In [441]:
print(bus.shape)
print(subway.shape)

(1575, 2)
(1575, 2)


In [442]:
bus.dropna(axis=0, how='any', inplace=True)
subway.dropna(axis=0, how='any', inplace=True)

In [443]:
print(bus.shape)
print(subway.shape)

(1281, 2)
(196, 2)


In [444]:
facility['상권_코드'].value_counts()

2110939    1
2110782    1
2110889    1
2110990    1
2110699    1
          ..
2110565    1
2120150    1
2110533    1
2130181    1
2110480    1
Name: 상권_코드, Length: 1575, dtype: int64

In [445]:
bus['상권_코드'].value_counts()

2110939    1
2120181    1
2110380    1
2110510    1
2110522    1
          ..
2110331    1
2110336    1
2110535    1
2110529    1
2110480    1
Name: 상권_코드, Length: 1281, dtype: int64

In [446]:
subway['상권_코드'].value_counts()

2130028    1
2120244    1
2130313    1
2120149    1
2120178    1
          ..
2120125    1
2120185    1
2120187    1
2120121    1
2120112    1
Name: 상권_코드, Length: 196, dtype: int64

In [447]:
# bus = bus.groupby(['상권_코드'], as_index=False).sum()
# subway = subway.groupby(['상권_코드'], as_index=False).sum()

In [448]:
print(bus.shape)
print(subway.shape)

(1281, 2)
(196, 2)


In [460]:
subway.isnull().sum()

상권_코드      0
지하철_역_수    0
dtype: int64

In [449]:
com_bus_list = [info, area, bus]
com_subway_list = [info, area, subway]

In [450]:
df_bus_1 = reduce(lambda  left,right: pd.merge(left,right,on=['상권_코드'], how='outer'), com_bus_list[1:])
df_subway_1 = reduce(lambda  left,right: pd.merge(left,right,on=['상권_코드'], how='outer'), com_subway_list[1:])


In [451]:
print(df_bus_1.shape)
print(df_subway_1.shape)

(1671, 8)
(1671, 8)


In [452]:
area.shape

(1671, 7)

In [453]:
df_bus_1

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,엑스좌표_값,와이좌표_값,시군구_코드,버스_정거장_수
0,202112,A,골목상권,2110753,191934,438813,11545,10.0
1,202112,A,골목상권,2110427,206265,459431,11350,1.0
2,202112,A,골목상권,2110755,192300,439070,11545,6.0
3,202112,A,골목상권,2110739,191289,438014,11545,3.0
4,202112,A,골목상권,2111017,210222,445516,11710,1.0
...,...,...,...,...,...,...,...,...
1666,202112,U,관광특구,1001493,200974,451975,11140,18.0
1667,202112,U,관광특구,1001494,199768,452273,11110,27.0
1668,202112,U,관광특구,1001495,210191,446339,11710,20.0
1669,202112,U,관광특구,1001496,205310,445727,11680,8.0


In [454]:
df_subway_1

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,엑스좌표_값,와이좌표_값,시군구_코드,지하철_역_수
0,202112,A,골목상권,2110753,191934,438813,11545,NaN
1,202112,A,골목상권,2110427,206265,459431,11350,NaN
2,202112,A,골목상권,2110755,192300,439070,11545,NaN
3,202112,A,골목상권,2110739,191289,438014,11545,NaN
4,202112,A,골목상권,2111017,210222,445516,11710,NaN
...,...,...,...,...,...,...,...,...
1666,202112,U,관광특구,1001493,200974,451975,11140,5.0
1667,202112,U,관광특구,1001494,199768,452273,11110,4.0
1668,202112,U,관광특구,1001495,210191,446339,11710,3.0
1669,202112,U,관광특구,1001496,205310,445727,11680,1.0


In [455]:
info = info[['시군구코드', '시군구명']]
info = info.drop_duplicates().reset_index(drop=True)
info.rename(columns={'시군구코드':'시군구_코드'}, inplace=True)

In [456]:
df_bus = pd.merge(df_bus_1, info, on='시군구_코드')
df_subway = pd.merge(df_subway_1, info, on='시군구_코드')

In [457]:
print(df_bus.shape)
df_bus

(1671, 9)


,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,엑스좌표_값,와이좌표_값,시군구_코드,버스_정거장_수,시군구명
0,202112,A,골목상권,2110753,191934,438813,11545,10.0,금천구
1,202112,A,골목상권,2110755,192300,439070,11545,6.0,금천구
2,202112,A,골목상권,2110739,191289,438014,11545,3.0,금천구
3,202112,A,골목상권,2110734,191061,439380,11545,NaN,금천구
4,202112,A,골목상권,2110751,191810,442086,11545,NaN,금천구
...,...,...,...,...,...,...,...,...,...
1666,202112,R,전통시장,2130268,195759,444396,11590,3.0,동작구
1667,202112,R,전통시장,2130271,197572,442683,11590,3.0,동작구
1668,202112,R,전통시장,2130270,196727,445276,11590,NaN,동작구
1669,202112,R,전통시장,2130269,195874,444157,11590,NaN,동작구


In [458]:
print(df_subway.shape)
df_subway

(1671, 9)


,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,엑스좌표_값,와이좌표_값,시군구_코드,지하철_역_수,시군구명
0,202112,A,골목상권,2110753,191934,438813,11545,NaN,금천구
1,202112,A,골목상권,2110755,192300,439070,11545,NaN,금천구
2,202112,A,골목상권,2110739,191289,438014,11545,NaN,금천구
3,202112,A,골목상권,2110734,191061,439380,11545,NaN,금천구
4,202112,A,골목상권,2110751,191810,442086,11545,NaN,금천구
...,...,...,...,...,...,...,...,...,...
1666,202112,R,전통시장,2130268,195759,444396,11590,NaN,동작구
1667,202112,R,전통시장,2130271,197572,442683,11590,NaN,동작구
1668,202112,R,전통시장,2130270,196727,445276,11590,NaN,동작구
1669,202112,R,전통시장,2130269,195874,444157,11590,NaN,동작구


In [459]:
df_bus.fillna(0, inplace=True)
df_subway.fillna(0, inplace=True)

In [421]:
df_bus

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,엑스좌표_값,와이좌표_값,시군구_코드,버스_정거장_수,시군구명
0,202112,A,골목상권,2110753,191934,438813,11545,10.0,금천구
1,202112,A,골목상권,2110755,192300,439070,11545,6.0,금천구
2,202112,A,골목상권,2110739,191289,438014,11545,3.0,금천구
3,202112,A,골목상권,2110734,191061,439380,11545,0.0,금천구
4,202112,A,골목상권,2110751,191810,442086,11545,0.0,금천구
...,...,...,...,...,...,...,...,...,...
1666,202112,R,전통시장,2130268,195759,444396,11590,3.0,동작구
1667,202112,R,전통시장,2130271,197572,442683,11590,3.0,동작구
1668,202112,R,전통시장,2130270,196727,445276,11590,0.0,동작구
1669,202112,R,전통시장,2130269,195874,444157,11590,0.0,동작구


In [422]:
df_subway

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,엑스좌표_값,와이좌표_값,시군구_코드,지하철_역_수,시군구명
0,202112,A,골목상권,2110753,191934,438813,11545,0.0,금천구
1,202112,A,골목상권,2110755,192300,439070,11545,0.0,금천구
2,202112,A,골목상권,2110739,191289,438014,11545,0.0,금천구
3,202112,A,골목상권,2110734,191061,439380,11545,0.0,금천구
4,202112,A,골목상권,2110751,191810,442086,11545,0.0,금천구
...,...,...,...,...,...,...,...,...,...
1666,202112,R,전통시장,2130268,195759,444396,11590,0.0,동작구
1667,202112,R,전통시장,2130271,197572,442683,11590,0.0,동작구
1668,202112,R,전통시장,2130270,196727,445276,11590,0.0,동작구
1669,202112,R,전통시장,2130269,195874,444157,11590,0.0,동작구


# 시군구명 버스 / 지하철 Groupby Sum

In [491]:
df_bus = df_bus.groupby(['시군구명'], as_index=False).sum()
df_bus = df_bus.sort_values(by='버스_정거장_수', ascending=True)
df_bus

,시군구명,기준_년월_코드,상권_코드,엑스좌표_값,와이좌표_값,시군구_코드,버스_정거장_수
18,양천구,11318272,118505901,10468350,25069556,642320,128.0
1,강동구,12126720,126910829,12735070,26950557,704400,162.0
8,노원구,8488704,88823932,8656946,19345596,476700,162.0
5,광진구,12530944,131138081,12845436,27880500,695330,191.0
17,송파구,14147840,146953779,14747188,31131636,819700,198.0
24,중랑구,12328832,128983495,12666497,27766464,686860,201.0
9,도봉구,8690816,90954284,8734377,19873523,486760,201.0
15,성동구,11116160,116325639,11189722,24772519,616000,206.0
10,동대문구,15562624,162942794,15737576,34911538,864710,206.0
2,강북구,12733056,133278426,12720976,28935350,712215,216.0


In [490]:
df_subway = df_subway.groupby(['시군구명'], as_index=False).sum()
df_subway = df_subway.sort_values(by='지하철_역_수', ascending=True)
df_subway

,시군구명,기준_년월_코드,상권_코드,엑스좌표_값,와이좌표_값,시군구_코드,지하철_역_수
4,관악구,14956288,156640334,14366816,32722691,859880,3.0
9,도봉구,8690816,90954284,8734377,19873523,486760,3.0
18,양천구,11318272,118505901,10468350,25069556,642320,4.0
7,금천구,9095040,95176793,8603407,19813101,519525,4.0
13,서대문구,12530944,131116143,12058425,28066028,707420,5.0
2,강북구,12733056,133278426,12720976,28935350,712215,6.0
16,성북구,15360512,160679682,15391501,34643234,858040,7.0
21,은평구,13743616,143805898,13111561,31028544,773840,8.0
15,성동구,11116160,116325639,11189722,24772519,616000,8.0
20,용산구,11520384,119375506,11304590,25565727,636690,9.0


# Folium 지도 시각화

In [467]:
import folium
import requests
import json
from folium.plugins import MarkerCluster
from branca.colormap import linear
from folium import plugins


# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [494]:
# map = folium.plugins.DualMap(location=[37.541, 126.986], zoom_start = 11, tiles=None)
map = folium.Map(location=[37.541, 126.986], zoom_start = 11, tiles='CartoDB positron')

# tiles='CartoDB positron'

# folium.TileLayer('CartoDB positron').add_to(map.m1)
folium.TileLayer('CartoDB positron').add_to(map)

# folium.TileLayer('Cloudmade').add_to(map.m1)

# folium.TileLayer('CartoDB positron').add_to(map.m2)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '서울시 지하철역 수',
    data = df_subway[['시군구명', '지하철_역_수']],
    columns = ['시군구명','지하철_역_수'],
    fill_color='PuBu', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="지역구별 지하철역 수",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '서울시 버스정거장 수',
    data = df_bus[['시군구명', '버스_정거장_수']],
    columns = ['시군구명','버스_정거장_수'],
    fill_color='PuBu', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="지역구별 버스정거장 수",
    overlay = True,
    Highlight = True
).add_to(map)


folium.LayerControl(collapsed = False).add_to(map)


## color
#YIGn
#Blues
#BuPu
#PuRd
#YlOrRd
#YlGn

map.save('지하철역_버스정거장_map.html')

map